# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [2]:
import pandas as pd

# Cargar el archivo desde el enlace
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
data = pd.read_csv(url)

# Visualizar las primeras filas para entender el formato
print(data.head())


   Unnamed: 0 Customer       State  Customer Lifetime Value Response  \
0           0  DK49336     Arizona              4809.216960       No   
1           1  KX64629  California              2228.525238       No   
2           2  LZ68649  Washington             14947.917300       No   
3           3  XL78013      Oregon             22332.439460      Yes   
4           4  QA50777      Oregon              9025.067525       No   

   Coverage Education Effective To Date EmploymentStatus Gender  ...  \
0     Basic   College           2/18/11         Employed      M  ...   
1     Basic   College           1/18/11       Unemployed      F  ...   
2     Basic  Bachelor           2/10/11         Employed      M  ...   
3  Extended   College           1/11/11         Employed      M  ...   
4   Premium  Bachelor           1/17/11    Medical Leave      F  ...   

   Number of Open Complaints Number of Policies     Policy Type        Policy  \
0                        0.0                  9  Corp

In [3]:
# Exercise 1

# Crear el nuevo DataFrame con las condiciones dadas
filtered_df = data[(data['Total Claim Amount'] > 1000) & (data['Response'] == "Yes")]

# Mostrar las primeras filas del nuevo DataFrame
print(filtered_df.head())



     Unnamed: 0 Customer       State  Customer Lifetime Value Response  \
189         189  OK31456  California             11009.130490      Yes   
236         236  YJ16163      Oregon             11009.130490      Yes   
419         419  GW43195      Oregon             25807.063000      Yes   
442         442  IP94270     Arizona             13736.132500      Yes   
587         587  FJ28407  California              5619.689084      Yes   

     Coverage             Education Effective To Date EmploymentStatus Gender  \
189   Premium              Bachelor           1/24/11         Employed      F   
236   Premium              Bachelor           1/24/11         Employed      F   
419  Extended               College           2/13/11         Employed      F   
442   Premium                Master           2/13/11         Disabled      F   
587   Premium  High School or Below           1/26/11       Unemployed      M   

     ...  Number of Open Complaints Number of Policies     Policy Ty

In [4]:
#Exercise 2

# Filtrar los datos para incluir solo las respuestas "Yes"
yes_responses = data[data['Response'] == "Yes"]

# Calcular el promedio de Total Claim Amount agrupado por Policy Type y Gender
average_claim = yes_responses.groupby(['Policy Type', 'Gender'])['Total Claim Amount'].mean().reset_index()

# Mostrar los resultados
print(average_claim)


      Policy Type Gender  Total Claim Amount
0  Corporate Auto      F          433.738499
1  Corporate Auto      M          408.582459
2   Personal Auto      F          452.965929
3   Personal Auto      M          457.010178
4    Special Auto      F          453.280164
5    Special Auto      M          429.527942


In [5]:
#Exercise 3

# Contar el número total de clientes por estado
customers_by_state = data['State'].value_counts().reset_index()
customers_by_state.columns = ['State', 'Customer Count']

# Filtrar estados con más de 500 clientes
filtered_states = customers_by_state[customers_by_state['Customer Count'] > 500]

# Mostrar los resultados
print(filtered_states)


        State  Customer Count
0  California            3552
1      Oregon            2909
2     Arizona            1937
3      Nevada             993
4  Washington             888


In [6]:
#Exercise 4

# Calcular las métricas requeridas
clv_stats = data.groupby(['Education', 'Gender'])['Customer Lifetime Value'].agg(['max', 'min', 'median']).reset_index()

# Mostrar los resultados
print(clv_stats)


              Education Gender          max          min       median
0              Bachelor      F  73225.95652  1904.000852  5640.505303
1              Bachelor      M  67907.27050  1898.007675  5548.031892
2               College      F  61850.18803  1898.683686  5623.611187
3               College      M  61134.68307  1918.119700  6005.847375
4                Doctor      F  44856.11397  2395.570000  5332.462694
5                Doctor      M  32677.34284  2267.604038  5577.669457
6  High School or Below      F  55277.44589  2144.921535  6039.553187
7  High School or Below      M  83325.38119  1940.981221  6286.731006
8                Master      F  51016.06704  2417.777032  5729.855012
9                Master      M  50568.25912  2272.307310  5579.099207


In [7]:
#BONUS - Exercise 5

# Asegurarnos de que las fechas estén en un formato adecuado
data['Effective To Date'] = pd.to_datetime(data['Effective To Date'])

# Extraer el mes del campo de la fecha
data['Month'] = data['Effective To Date'].dt.month

# Crear una tabla pivote con estados como filas y meses como columnas
policies_by_state_month = data.pivot_table(
    index='State', 
    columns='Month', 
    values='Policy',  # Usamos cualquier columna que represente las pólizas (puedes ajustar si hay otra mejor)
    aggfunc='count', 
    fill_value=0  # Rellenar valores faltantes con 0
)

# Mostrar la tabla pivote
print(policies_by_state_month)


Month          1     2
State                 
Arizona     1008   929
California  1918  1634
Nevada       551   442
Oregon      1565  1344
Washington   463   425


C:\Users\cdeniaca\AppData\Local\Temp\ipykernel_23120\1210193294.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['Effective To Date'] = pd.to_datetime(data['Effective To Date'])


In [8]:
#Exercise 6

# Asegurarnos de que las fechas estén en un formato adecuado
data['Effective To Date'] = pd.to_datetime(data['Effective To Date'])

# Extraer el mes del campo de la fecha
data['Month'] = data['Effective To Date'].dt.month

# Agrupar por estado y mes, y contar el número de pólizas vendidas
policies_by_state_month = data.groupby(['State', 'Month'])['Policy'].count().reset_index()
policies_by_state_month.columns = ['State', 'Month', 'Policy Count']

# Calcular el número total de pólizas por estado
total_policies_by_state = policies_by_state_month.groupby('State')['Policy Count'].sum().reset_index()
total_policies_by_state = total_policies_by_state.sort_values(by='Policy Count', ascending=False)

# Seleccionar los 3 estados con el mayor número de pólizas vendidas
top_3_states = total_policies_by_state.head(3)['State']

# Filtrar los datos para incluir solo los 3 estados principales
filtered_data = policies_by_state_month[policies_by_state_month['State'].isin(top_3_states)]

# Pivotar la tabla para mostrar los meses como columnas y los estados como filas
top_3_states_table = filtered_data.pivot_table(
    index='State',
    columns='Month',
    values='Policy Count',
    aggfunc='sum',
    fill_value=0
)

# Mostrar la tabla final
print(top_3_states_table)


Month          1     2
State                 
Arizona     1008   929
California  1918  1634
Oregon      1565  1344


In [9]:
#Exercise 7

# Seleccionar las columnas relevantes
marketing_channels = ['Response', 'Sales Channel', 'State', 'Policy Type', 'Gender']  # Ajusta si necesitas otras columnas relevantes
selected_data = data[marketing_channels]

# Filtrar solo las respuestas "Yes"
response_yes = selected_data[selected_data['Response'] == "Yes"]

# Calcular el número total de clientes por canal
total_by_channel = data.groupby('Sales Channel')['Response'].count().reset_index()
total_by_channel.columns = ['Sales Channel', 'Total Customers']

# Calcular el número de respuestas "Yes" por canal
yes_by_channel = response_yes.groupby('Sales Channel')['Response'].count().reset_index()
yes_by_channel.columns = ['Sales Channel', 'Yes Responses']

# Unir los DataFrames para calcular la tasa de respuesta
response_rate = pd.merge(yes_by_channel, total_by_channel, on='Sales Channel')
response_rate['Response Rate (%)'] = (response_rate['Yes Responses'] / response_rate['Total Customers']) * 100

# Mostrar los resultados
print(response_rate)


  Sales Channel  Yes Responses  Total Customers  Response Rate (%)
0         Agent            742             3890          19.074550
1        Branch            326             2865          11.378709
2   Call Center            221             2013          10.978639
3           Web            177             1511          11.714097
